In [ ]:
#Plot Histograms

binNum    = [0, 75, 150, 225, 300, 375, 450, 525, 600, 675, 750, 825, 900]
#binNum    = [0, 25,50,75, 100,150, 200, 250, 300, 350, 400, 450, 500, 550,600, 650,700, 750, 800,850, 900]
minorTicks = [0, 50, 100, 200, 250, 300, 350, 450, 500, 550,600, 700, 750, 800,850]
majorTicks = [150,400,650,900]
# minorTicks = [0,75, 225, 300, 375, 450, 600, 675, 750, 825]
# majorTicks = [150, 525, 900]
color1, color2, color3 = 'deepskyblue', 'magenta', 'black'
label1 = 'From SNe'
label3 = 'Beam Radius'
fsize = 18
fig, axs = plt.subplots(2, 5, sharex=False, sharey = True, tight_layout=True)
fig.set_figheight(6)
fig.set_figwidth(13)
plt.tick_params(axis='y', direction = 'in')

axs[0,0].set_title(r'$10^{5.5} M_{Sun}$', fontsize = fsize)
axs[0,0].text(300, 40, 'AlphaCO = 6.7', fontsize=14)
axs[0,0].hist(nearestMCSN_55_int, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[0,0].axvline(75, color=color3, label = label3)
axs[0,0].set_xticks(ticks=majorTicks)
axs[0,0].set_xticks(ticks=minorTicks, minor=True)
axs[0,0].tick_params(axis='y', direction = 'in')

axs[0,1].set_title(r'$10^{6} M_{Sun}$', fontsize = fsize)
axs[0,1].hist(nearestMCSN_6_int, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[0,1].axvline(75, color=color3, label = label3)
axs[0,1].set_xticks(ticks=majorTicks)
axs[0,1].set_xticks(ticks=minorTicks, minor=True)
axs[0,1].tick_params(axis='y', direction = 'in')

axs[0,2].set_title(r'$10^{6.5} M_{Sun}$', fontsize = fsize)
axs[0,2].hist(nearestMCSN_65_int, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[0,2].axvline(75, color=color3, label = label3)
axs[0,2].set_xticks(ticks=majorTicks)
axs[0,2].set_xticks(ticks=minorTicks, minor=True)
axs[0,2].tick_params(axis='y', direction = 'in')

axs[0,3].set_title(r'$10^{7} M_{Sun}$', fontsize = fsize)
axs[0,3].hist(nearestMCSN_7_int, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[0,3].axvline(75, color=color3, label = label3)
axs[0,3].set_xticks(ticks=majorTicks)
axs[0,3].set_xticks(ticks=minorTicks, minor=True)
axs[0,3].tick_params(axis='y', direction = 'in')

axs[0,4].set_title(r'$10^{7.5} M_{Sun}$', fontsize = fsize)
axs[0,4].hist(nearestMCSN_75_int, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[0,4].axvline(75, color=color3, label = label3)
axs[0,4].set_xticks(ticks=majorTicks)
axs[0,4].set_xticks(ticks=minorTicks, minor=True)
axs[0,4].tick_params(axis='y', direction = 'in')

axs[1,0].text(425, 26, 'AlphaCO', fontsize=14)
axs[1,0].text(425, 40, 'Metallicity', fontsize=14)
axs[1,0].text(425, 33, 'Dependent', fontsize=14)
axs[1,0].hist(nearestMCSN_55_mass, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[1,0].axvline(75, color=color3, label = label3)
axs[1,0].set_xticks(ticks=majorTicks)
axs[1,0].set_xticks(ticks=minorTicks, minor=True)
axs[1,0].tick_params(axis='y', direction = 'in')

axs[1,1].hist(nearestMCSN_6_mass, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[1,1].axvline(75, color=color3, label = label3)
axs[1,1].set_xticks(ticks=majorTicks)
axs[1,1].set_xticks(ticks=minorTicks, minor=True)
axs[1,1].tick_params(axis='y', direction = 'in')

axs[1,2].hist(nearestMCSN_65_mass, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[1,2].axvline(75, color=color3, label = label3)
axs[1,2].set_xticks(ticks=majorTicks)
axs[1,2].set_xticks(ticks=minorTicks, minor=True)
axs[1,2].tick_params(axis='y', direction = 'in')

axs[1,3].hist(nearestMCSN_7_mass, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[1,3].axvline(75, color=color3, label = label3)
axs[1,3].set_xticks(ticks=majorTicks)
axs[1,3].set_xticks(ticks=minorTicks, minor=True)
axs[1,3].tick_params(axis='y', direction = 'in')

axs[1,4].hist(nearestMCSN_75_mass, bins=binNum,  color = color1, edgecolor='black', label = label1)
axs[1,4].axvline(75, color=color3, label = label3)
axs[1,4].set_xticks(ticks=majorTicks)
axs[1,4].set_xticks(ticks=minorTicks, minor=True)
axs[1,4].tick_params(axis='y', direction = 'in')

# axs[0,0].legend(loc='best', fontsize = 14)
# axs[0,1].legend(loc='best', fontsize = 14)
# axs[0,2].legend(loc='best', fontsize = 14)
# axs[0,3].legend(loc='best', fontsize = 14)
axs[0,4].legend(loc='best', fontsize = 14)
# axs[1,0].legend(loc='best', fontsize = 14)
# axs[1,1].legend(loc='best', fontsize = 14)
# axs[1,2].legend(loc='best', fontsize = 14)
# axs[1,3].legend(loc='best', fontsize = 14)
# axs[1,4].legend(loc='best', fontsize = 14)

axs[1,0].set_ylabel('Number of Molecular Clouds', fontsize = 20)
axs[1,2].set_xlabel('Distance (pc)', fontsize = 20)

fig.savefig('../Figures/Hist_nearestMCs_withRand.png')


In [ ]:
#Plot Histograms

binNum = 20

fig, axs = plt.subplots(1, 5, sharex=False, sharey = True, tight_layout=True)
fig.set_figheight(3)
fig.set_figwidth(13)
plt.tick_params(axis='y', direction = 'in')

axs[0].hist(nearestMC_55, bins=binNum, edgecolor='black', label = r'SNR $>$ 1')
axs[0].set_xticks(ticks=[100, 300, 500, 700, 900])
axs[0].set_xticks(ticks=[0, 200, 400, 800, 1000], minor=True)
axs[0].tick_params(axis='y', direction = 'in')

axs[1].hist(nearestMC_6, bins=binNum, edgecolor='black', label = r'SNR $>$ 3')
axs[1].set_xticks(ticks=[100, 300, 500, 700, 900])
axs[1].set_xticks(ticks=[0, 200, 400, 800, 1000], minor=True)
axs[1].tick_params(axis='y', direction = 'in')

axs[2].hist(nearestMC_65, bins=binNum, edgecolor='black', label = r'SNR $>$ 5')
axs[2].set_xticks(ticks=[100, 300, 500, 700, 900])
axs[2].set_xticks(ticks=[0, 200, 400, 800, 1000], minor=True)
axs[2].tick_params(axis='y', direction = 'in')

axs[3].hist(nearestMC_7, bins=binNum, edgecolor='black', label = r'SNR $>$ 70')
axs[3].set_xticks(ticks=[100, 300, 500, 700, 900])
axs[3].set_xticks(ticks=[0, 200, 400, 800, 1000], minor=True)
axs[3].tick_params(axis='y', direction = 'in')

axs[4].hist(nearestMC_75, bins=binNum, edgecolor='black', label = r'SNR $>$ 200')
axs[4].set_xticks(ticks=[100, 300, 500, 700, 900])
axs[4].set_xticks(ticks=[0, 200, 400, 800, 1000], minor=True)
axs[4].tick_params(axis='y', direction = 'in')

axs[0].legend(loc='best', fontsize = 16)
axs[1].legend(loc='best', fontsize = 16)
axs[2].legend(loc='best', fontsize = 16)
axs[3].legend(loc='best', fontsize = 16)
axs[4].legend(loc='best', fontsize = 16)


axs[0].set_ylabel('Number of Supernova', fontsize = 20)
axs[2].set_xlabel('Distance (pc)', fontsize = 20)

fig.savefig('../Figures/Hist_nearestMCs_SNR.png')
print(len(nearestMC_55), len(nearestMC_6), len(nearestMC_65), len(nearestMC_7), len(nearestMC_75))